In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
#load cvs file to a DataFrame
csv_file = "output_data/cities.csv"

data_frame = pd.read_csv(csv_file)
data_frame.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [3]:
# Configure gmaps.
gmaps.configure(api_key=g_key)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = data_frame[['Lat', 'Lng']]
weights = data_frame['Humidity']
fig = gmaps.figure()

# Add Heatmap layer to map.
heatmap = fig.add_layer(gmaps.heatmap_layer(locations, weights=weights,))
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
data_frame

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


In [6]:
# Narrow down the cities to fit weather conditions.
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Drop any rows will null values.

cities_weather = data_frame[
    (data_frame['Max Temp'] > 70) & 
    (data_frame['Max Temp'] < 80) &
    (data_frame['Wind Speed'] < 10.00) &
    (data_frame['Cloudiness'] == 0)
]
cities_weather

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


# Hotel Map

In [7]:
# Store into variable named hotel_df.
hotel_df = pd.DataFrame(data=cities_weather)

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"]= ""

# Set parameters to search for hotels with 5000 meters.
params= {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Hit the Google Places API for each city's coordinates.
hotel_df.loc[:,"Coordinates"] = hotel_df.loc[:,"Lat"].astype(str) +"," + hotel_df.loc[:,"Lng"].astype(str)

hotel_list = []

for location in hotel_df["Coordinates"]:

    params['location'] = location

    gmaps_json = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params=params).json()
    print(gmaps_json)
    
    try:
        hotel_list.append(gmaps_json["results"][0]["name"])
        
    except:
        hotel_list.append("N/A")
        
hotel_df["Hotel Name"] = hotel_list
hotel_df.loc[:, ~hotel_df.columns.str.contains('^Unnamed')]
# # Store the first Hotel result into the DataFrame.
# # Plot markers on top of the heatmap.

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -18.054129, 'lng': 13.8438893}, 'viewport': {'northeast': {'lat': -18.0526479197085, 'lng': 13.8452797802915}, 'southwest': {'lat': -18.0553458802915, 'lng': 13.8425818197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': 'f5d4048923aea3593f0e849856ab1f24a5ce07bc', 'name': 'ABBA Guesthouse', 'opening_hours': {'open_now': False}, 'photos': [{'height': 1536, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103313244361034711823">ABBA Guesthouse</a>'], 'photo_reference': 'CmRaAAAAXOP03fPohbV8xxI8OXlTQjuO_-58zoMMTDjCpI7fzDSdN4OZSzRT0sTWzh4oQqdSOt0fHGQtyXeKEcO5eUpLKkU6EvUV3NxI-TO4hKayyRrWlBze05vvVdyNL7WDLU6vEhAHah2UXpkufI1qxNAaBqeAGhSL2USK9UeSQiNifBIZhOJ7tQmeqA', 'width': 2048}], 'place_id': 'ChIJa15j5MeTkBsRFpqaXwd1FXg', 'plus_code': {'compound_code': 'WRWV+8H Opuwo, Namibia', 'global_code': '5FHMWRWV+8H'}, 'rating': 3.9, 'refe

{'html_attributions': [], 'next_page_token': 'CqQCHAEAALbw9pnK9RLkvDIevV3NZ3Z-6vm9MJC_MQrSj-YjVEcl5QwO3uHxEGfvdL0UcWRlH_B72kdG-dzRQjQwL1Uy_StaOo2vu-HlOqd1zQ2lANuCvdyIsefHkK2AnBoic9KsSvwtfZRV8WJXll-HY-uXkaU1Fj9Keyyz0nsXz8a4JAwKTvTx-uqDG-dfhlnN-agGXVl2-EVbQvBHK1Lu5JXIn4aPmMGqfkVbYsVMPMNzH6BVDJmpStwkvt4GXvZlQEscyoxLFKB3Ej9WaWMBgtYsl1Ru7LwND5GJLj6rRjRUe91LcL7u_20V5x7IrFLCHWxDtgt75hWCcNQ3coqrtIBLPUcuSbou69Prim9pn5XVCuJaPsIM1yYlW_SVNhsqB0re8BIQwlgMYTt2WnIHCVk_6tOq8xoUmWXziUFjs0emCGLn3ZSGU9gEw9I', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -20.6535749, 'lng': -46.0153976}, 'viewport': {'northeast': {'lat': -20.6522770197085, 'lng': -46.0140075697085}, 'southwest': {'lat': -20.6549749802915, 'lng': -46.0167055302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': 'bac7123a39f66d774eda4d96d8affa6cec4d595d', 'name': 'Hotel Escarpas do Lago', 'opening_hours': {'open_now': False}, 'photos': [{'height': 960, 'html_attributio

{'html_attributions': [], 'next_page_token': 'CqQCHAEAACo1Dm_0ZOG591uC9t5yW70DteiPmhLRjj785OnD3e_XVaWCrq96ndaP-MSORKks5joidSG6mRUFDe9O2DEOPNKublTMb21162pA5R-ObZnXgRSROS_szJZAzGWXCUNlB7jxz7nTvqSeVMQTeaIqV8kFurz0u3MsUl9adqUaXVWR08izHHgESGWIVjp-JO89exQXONK0r2qC1DXl8KCqOLCcFxS5DfrdO0giBhKq4p4Q7y9KUYHX0cuVCcbjHY9bkoreth3SH6MHT4wYD3oRSI0fEq8jreGn4j0GS16BN_K1DT8FGv5Eg3nrXFfHfSOLim2IjlbWaFnB2qjnUmc2nG9yBP7x0Y3_8dygaMvZP3HmrMoIA5ZOWa_b3-U24GmUyRIQFmycj0Wy-SARqwKMsivwyxoUjHw1gGXnGNOwT7_S3SinbS_ZNoc', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -30.1592308, 'lng': -50.1997331}, 'viewport': {'northeast': {'lat': -30.1578710697085, 'lng': -50.1984219697085}, 'southwest': {'lat': -30.16056903029149, 'lng': -50.2011199302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': 'b7df7ea63e23d1fe48b1dc74920fbdd2c37fbe59', 'name': 'Novomar Pousada e Turismo Ltda', 'photos': [{'height': 2448, 'html_attributions': ['<a href="https://maps

{'html_attributions': [], 'next_page_token': 'CqQCHAEAAHVTbqrFFSQ4MFmnOs9MAYKg9dVaV_iA9t6sf0vyNigajdo9qCAdcqbnbSrfTqIXNhlJcMc5glqWZXT7dxt__lvbbqBtU6Qn6GdeZRyuP8eJRaAwgNU4LtSIx40YJtKiILyngS-g2Jm8THTpgyE62oFNCbcz9S-zdWFMKa-wN4TmgcYbJP8FpaVPhtN8cEoLd3pkEbj_2PIvYeAU7Jt7eBT0ybieiuM8oUjHrUW9UI6LzqNEuQ2m1EcfsnOsqslcqYE4AXbZoPxrjA3PqugNwA1mvWhSX_KlONS_v9uyA5kbzCejrLx1N7pFcYQU0WvRKHIvTJYC2zmxEfPFzYZcqjrD1QauajmRdx-XpjVKWrUptmgNpinkcIv0JYA7oB75cxIQYVqmxfqDIzpnx9qZvDFLVBoUMW_13rxxGT7hIp2Glx86pWn7k7M', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -20.2979423, 'lng': 44.2694266}, 'viewport': {'northeast': {'lat': -20.2965964197085, 'lng': 44.2707963302915}, 'southwest': {'lat': -20.2992943802915, 'lng': 44.2680983697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '6d24a15b2f45c292f3b0e4ad73ccaef84135def2', 'name': 'La Case Bambou', 'place_id': 'ChIJM7-2q0vQXh8RGYHnjh5k4mA', 'plus_code': {'compound_code': 'P729+RQ Morondava

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 15.3232474, 'lng': 74.5068496}, 'viewport': {'northeast': {'lat': 15.3245984802915, 'lng': 74.50811953029151}, 'southwest': {'lat': 15.3219005197085, 'lng': 74.50542156970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '5346a344f6764c5be11c9d1832214eecd45eed07', 'name': 'Dandeli Lake County', 'opening_hours': {'open_now': True}, 'photos': [{'height': 853, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/100842033022261574539">Dandeli Lake County</a>'], 'photo_reference': 'CmRaAAAAjE7-xgADm6MRKEBp24uL9q6WH1BqwHlK5jHmFV729nVoe9-5ca490YScCfpvMPBhA3APt5uxpc8CHlF90wAlnEp-Jpy7FM3jyxibTwJ-RawkOhFQu9YNTYDC8UCx7lJUEhB1ms0PyPmhqbFum2gfHwAeGhT4jH8NvLYAtIDOUOU1bQYR1al0RA', 'width': 1280}], 'place_id': 'ChIJ2UWvXKwevzsRMuMzNgEdz_I', 'plus_code': {'compound_code': '8GF4+7P Jagalbet, Karnataka, India', 'global_code': '7J7P8GF4+7P'}, 'r

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 22.9213747, 'lng': 96.4994529}, 'viewport': {'northeast': {'lat': 22.9227446802915, 'lng': 96.5008386802915}, 'southwest': {'lat': 22.92004671970849, 'lng': 96.49814071970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': 'cb4f0734b9ea6dcb878f3688fada322c7b1a1616', 'name': 'MOGOK HILL HOTEL', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3584, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115205834117584060853">Thin Zar</a>'], 'photo_reference': 'CmRaAAAA2rYa3J-9Zfz-oRFNwRaX6-Sy7Lcz3NRIjT59S4hmOgRVV2x1V9twx2EvyaRIOXfm309Ux5umVUGEO9rWsgMqsPFRVUeG_s18FgNd9HKdJuWE4yS8ULad8MFmez8vwHU0EhBHpzvLgOHP0ZKWNo2Zcm9GGhQFKpaEZ1KvJlT2qitDqV2au9z2Ng', 'width': 3968}], 'place_id': 'ChIJabLwttZmMzcRV8-ndatBJOE', 'plus_code': {'compound_code': 'WFCX+GQ Mogok, Myanmar (Burma)', 'global_code': '7MJRWFCX+GQ'}, 'rating': 3.9, 'ref

{'html_attributions': [], 'next_page_token': 'CqQCHAEAALik2mbnQ9e_kUD6DMrRaBtE7eh0xBfXI2d82_VtTUZMwiFDvvba081R-_l-PrDAAUcKExVpNKSGiugI5Wl9lK2kHElA0G3KQEM8A29Fnw2CxVCM8OUcKN8kXc0AB211IKwic3gKE1b48lq5ASR31T1klkpKRn3kV07eLEdK74HQruNu-sVsOOlCBW3yZfgguVbQ96atwf15VXLoHvEmHjrzRfwqvzX1BA2vKKtCqSuee02QQpcf-bV3IY9SodYX-P9OMTkVY_hfdQFljv1XOrvmsY-OpNnloTlOvN8Ow8kzGVByM1BXSuj7Fmy7ISAKv_9dG434Jc6bbpZz7vTIU2hP4totVlOAps0AZtWWdbH_fbVepDnAYCvvxJ4bX7QHXRIQ9NS18785M1egVjx3zW1UqRoUTwtheNK6NWyYDlLNtS3gBQT2Qug', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.8625177, 'lng': 59.2037759}, 'viewport': {'northeast': {'lat': 32.86386753029149, 'lng': 59.20508738029152}, 'southwest': {'lat': 32.86116956970849, 'lng': 59.2023894197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': 'eba57afc6c72812cbd19143cae3215ef8d007e80', 'name': 'تاکسی سرویس آرامش', 'place_id': 'ChIJnTu1F_tdGj8RpIt30JNB67Y', 'plus_code': {'compound_code': 'V673+2G Birjan

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Coordinates
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,ABBA Guesthouse,"-18.06,13.84"
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Hotel Escarpas do Lago,"-20.63,-46.0"
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Novomar Pousada e Turismo Ltda,"-30.17,-50.22"
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,La Case Bambou,"-20.3,44.28"
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Lake County,"15.34,74.49"
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel La Giralda,"35.17,-2.93"
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,MOGOK HILL HOTEL,"22.92,96.51"
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,تاکسی سرویس آرامش,"32.86,59.22"


In [8]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Coordinates
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,ABBA Guesthouse,"-18.06,13.84"
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Hotel Escarpas do Lago,"-20.63,-46.0"
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Novomar Pousada e Turismo Ltda,"-30.17,-50.22"
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,La Case Bambou,"-20.3,44.28"
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Lake County,"15.34,74.49"
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel La Giralda,"35.17,-2.93"
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,MOGOK HILL HOTEL,"22.92,96.51"
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,تاکسی سرویس آرامش,"32.86,59.22"


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
new_fig = fig = gmaps.figure()

markers = gmaps.marker_layer(locations, hotel_info)
weights = hotel_df['Humidity']
new_fig.add_layer(gmaps.heatmap_layer(locations, weights=weights,))
new_fig.add_layer(markers)
new_fig

# Display figure

Figure(layout=FigureLayout(height='420px'))